In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
import os
import sys
import logging

try:
    import cv2
except ImportError:
    # on my raspi the opencv installation is only found here - but not in my virtualenv
    sys.path.insert(0,'/usr/local/lib/python3.5/dist-packages/cv2/python-3.5')
    
#from matplotlib import pyplot as plt
#from matplotlib.pyplot import imshow
#from PIL import Image
import tensorflow as tf
import numpy as np

logging.basicConfig(level=logging.DEBUG)


/home/pi/.python-tf-env/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/pi/.python-tf-env/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/pi/.python-tf-env/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/pi/.python-tf-env/lib/python3.5/site-packages/tensorflow/python/framework

In [3]:
# If you have the module installed, just use "import labinet..."
import inspect
# the .travis.yml is coded so that we execute tests from within test subdir. Relative to 'test' the .py is found in ../source/dev
cmd_subfolder = os.path.realpath(os.path.abspath(os.path.join(os.path.split(inspect.getfile(inspect.currentframe()))[0], "..")))
print('current folder is: {}'.format(cmd_subfolder))
if cmd_subfolder not in sys.path:
    sys.path.insert(0, cmd_subfolder)
    logging.debug('added to sys.path')
import labinet.box
import object_detection
import object_detection.protos
#import object_detection.protos.string_int_label_map_pb2
print('Import Done.')

current folder is: /home/pi/labinet
Import Done.


In [ ]:
print(sys.path)

In [4]:
from labinet.io_util import load_label_map
from labinet.io_util import load_model


In [5]:
%aimport labinet.io_util
%aimport labinet.object_detect
%aimport labinet.box


In [6]:
try:
    from labinet.io_util import load_label_map
    from labinet.io_util import load_model
    import labinet.object_detect
    import labinet.box
    logging.debug('Imported by default')
except ImportError:
    # This part is only required to run the notebook
    # directory when the module itself is not installed.
    #
    # If you have the module installed, just use "import labinet..."
    import inspect
    # the .travis.yml is coded so that we execute tests from within test subdir. Relative to 'test' the .py is found in ../source/dev
    cmd_subfolder = os.path.realpath(os.path.abspath(os.path.join(os.path.split(inspect.getfile(inspect.currentframe()))[0], "..")))
#    print('current folder is: {}'.format(cmd_subfolder))
    if cmd_subfolder not in sys.path:
        sys.path.insert(0, cmd_subfolder)
        logging.debug('added to sys.path')
    from labinet.io_util import load_label_map
    from labinet.io_util import load_model
    import labinet.object_detect
    import labinet.box

%aimport labinet.io_util
%aimport labinet.object_detect
%aimport labinet.box

%matplotlib inline

In [7]:
#print(sys.path)
#import object_detection.protos
# from object_detection.protos import string_int_label_map_pb2
import cv2

In [ ]:
video = cv2.VideoCapture(0)

In [ ]:
# Try capturing one frame
ret, frame = video.read()
if frame is None:
    print("Error - did you connect your webcam?")
else:
    cv2.imshow('object detection', cv2.resize(frame, (640,480)))
    key_pressed = cv2.waitKey(25)
    print(key_pressed)    

In [ ]:
# close the window again
cv2.destroyAllWindows()
video.release()

 ## Object Detection via Webcam
 Actually it is a loop to capture picture via webcam and to Object detection
 

In [8]:
# Paths to model etc pp
MODEL_NAME = 'inference_graph'   # the result from Step 6 Export Inference Graph
# Size, in inches, of the output images.
IMAGE_SIZE = (480, 640)
CWD_PATH = os.path.join(os.getcwd(),'..')  # should become gitbase
NUM_CLASSES = 1
# model
PATH_TO_MODEL = os.path.join(CWD_PATH, MODEL_NAME, 'frozen_inference_graph.pb')
# label map
LABEL_MAP = os.path.join(CWD_PATH, 'data', 'object-detection.pbtxt')


In [9]:
# load model and labels
detection_graph = load_model(PATH_TO_MODEL)
categories, category_index = load_label_map(LABEL_MAP, NUM_CLASSES)

W0728 12:14:08.421109 1996322368 deprecation_wrapper.py:119] From /home/pi/labinet/labinet/io_util.py:15: The name tf.GraphDef is deprecated. Please use tf.compat.v1.GraphDef instead.

W0728 12:14:08.427089 1996322368 deprecation_wrapper.py:119] From /home/pi/labinet/labinet/io_util.py:16: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



In [10]:
# prepare tensor dict for inference
tensor_dict = labinet.object_detect.get_tensor_dict_with_masks(IMAGE_SIZE[1], IMAGE_SIZE[0], detection_graph)
image_tensor = tensor_dict['image_tensor']
config = tf.ConfigProto()
#config.gpu_options.allow_growth=True

set: num_detections:0
set: detection_boxes:0
set: detection_scores:0
set: detection_classes:0
set: image_tensor:0
tensor:0 for image.shape=(x=640,y=480)


In [11]:
# Prepare the Cam!
video = cv2.VideoCapture(0)
ret, frame = video.read()
if frame is None:
    print("Error - did you connect your webcam?")

In [ ]:
%aimport labinet.object_detect
import cv2

In [ ]:
with tf.Session(graph=detection_graph, config=config) as sess:
    # capture
    while(True):
        ret, frame = video.read()
        #print(f"Captured frame.shape={frame.shape} - type(frame)={type(frame)}")
        image_np_exp = np.expand_dims(frame, axis=0) 
        #print(f'np-frame.shape={image_np_exp.shape}')
        # inference
        output_dict = sess.run(tensor_dict, feed_dict={image_tensor: image_np_exp})
        labinet.object_detect.convert_output_dict(output_dict)
        # visualize boxes
        image_with_boxes = labinet.object_detect.visualize_boxes_after_detection(frame, output_dict, category_index)
        # show image
        cv2.imshow('Detection Running...', cv2.resize(image_with_boxes,(IMAGE_SIZE[1],IMAGE_SIZE[0])))
        #cv2.waitKey(25)
        if cv2.waitKey(25) == ord('q'):
            break               
    


In [ ]:
video.release()
cv2.destroyAllWindows()